In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
############################################################
# pyupbit.get_current_price("KRW-BTC") : BIT 현재가
# --> price_KRW_BTC : BTC 현재가 
# upbit.get_balance(ticker="KRW-BTC") : BIT 잔고 조회 (My_Wallet_BIT)
# My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
# My_Wallet_Money = upbit.get_balance(ticker="KRW")
# My_Wallet_BITWON = float(price_KRW_BTC * My_Wallet_BIT))
# 
# print(upbit.get_balance("KRW-XRP"))     # KRW-XRP 조회
# print(upbit.get_balance("KRW"))         # 보유 현금 조회
# 
############################################################
import os
import jwt
import uuid
import hashlib
import pyupbit
import codecs
import datetime, time# for sleep
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from urllib.parse import urlencode
import codecs
import shutil
import math

import requests

from init_Stock import *

access_key = os.environ['UPBIT_OPEN_API_ACCESS_KEY']
secret_key = os.environ['UPBIT_OPEN_API_SECRET_KEY']
server_url = os.environ['UPBIT_OPEN_API_SERVER_URL']
upbit = pyupbit.Upbit(access_key, secret_key)

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
authorize_token = 'Bearer {}'.format(jwt_token)
headers = {"Authorization": authorize_token}

res = requests.get(server_url + "/v1/accounts", headers=headers)

# print(res.json())
# [{'currency': 'BTC', 'balance': '0.00026', 'locked': '0.0', 'avg_buy_price': '54041000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]

############################################################
# print("# Dataframe")
# print(Coin_Frame)
df = pd.DataFrame(Coin_Frame) # init_Stock.py에러 호출
# print(df)

# print("###########################")
# for index, row in df.iterrows(): # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)
#     print(index, row)
#     print(row['코인명'])

############################################################
# Market 조건
Coin_Buy_Condition_List, Coin_Sell_Condition_List = [0], [0]
for index, row in df.iterrows() : # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)
    if row['코인명'] != 'KRW' :
        Daily_Check_Coin = pyupbit.get_ohlcv(row['코인명'], interval="day", count=365)
        # print("#  -------------------------- #")
        # print(Daily_Check_Coin)
        # print("#  -------------------------- #")
        Daily_Check_Coin['diff1'] = round(((Daily_Check_Coin['high'] - Daily_Check_Coin['low']) / Daily_Check_Coin['low']) * 100,2)
        Daily_Check_Coin['diff2'] = round(((Daily_Check_Coin['open'] - Daily_Check_Coin['close']) / Daily_Check_Coin['open']) * 100,2)
        Daily_Check_Coin_Minus = Daily_Check_Coin[Daily_Check_Coin['diff2'] < 0]
        Daily_Check_Coin_Plus = Daily_Check_Coin[Daily_Check_Coin['diff2'] > 0]
        # print("# Coin 마이너스만..")
        # print(Daily_Check_Coin_Minus['diff2'].max())
        # print(Daily_Check_Coin_Minus['diff2'].min())
        # print(Daily_Check_Coin_Minus['diff2'].mean())
        Coin_Lower_Value_V = round(Daily_Check_Coin_Minus['diff2'].mean(),1)
        Coin_Upper_Value_V = round(Daily_Check_Coin_Plus['diff2'].mean(),1)
        # print("")
        Coin_Buy_Condition_List.append(Coin_Lower_Value_V)
        Coin_Sell_Condition_List.append(Coin_Upper_Value_V)
# print(Coin_Buy_Condition_List)
df['Buy_Condition'] = Coin_Buy_Condition_List
df['Sell_Condition'] = Coin_Sell_Condition_List
print(df)
print("##########################")
############################################################
# BTC 조건
Daily_Check_BTC = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365)
Daily_Check_BTC['diff1'] = round(((Daily_Check_BTC['high'] - Daily_Check_BTC['low']) / Daily_Check_BTC['low']) * 100,2)
Daily_Check_BTC['diff2'] = round(((Daily_Check_BTC['open'] - Daily_Check_BTC['close']) / Daily_Check_BTC['open']) * 100,2)
Daily_Check_BTC_Minus = Daily_Check_BTC[Daily_Check_BTC['diff2'] < 0]
print("# BTC 마이너스만..")
print(Daily_Check_BTC_Minus['diff2'].max())
print(Daily_Check_BTC_Minus['diff2'].min())
print(Daily_Check_BTC_Minus['diff2'].mean() * 2) # 엥???? 
BTC_Lower_Value_V = round(Daily_Check_BTC_Minus['diff2'].mean() * 2,1)
############################################################
# ETH 조건
Daily_Check_ETH = pyupbit.get_ohlcv("KRW-ETH", interval="day", count=365)
Daily_Check_ETH['diff1'] = round(((Daily_Check_ETH['high'] - Daily_Check_ETH['low']) / Daily_Check_ETH['low']) * 100,2)
Daily_Check_ETH['diff2'] = round(((Daily_Check_ETH['open'] - Daily_Check_ETH['close']) / Daily_Check_ETH['open']) * 100,2)
Daily_Check_ETH_Minus = Daily_Check_ETH[Daily_Check_ETH['diff2'] < 0]
print("# ETH 마이너스만..")
print(Daily_Check_ETH_Minus['diff2'].max())
print(Daily_Check_ETH_Minus['diff2'].min())
print(Daily_Check_ETH_Minus['diff2'].mean() * 2) # 엥???? 
ETH_Lower_Value_V = round(Daily_Check_ETH_Minus['diff2'].mean() * 2,1)
############################################################
# XRP 조건
Daily_Check_XRP = pyupbit.get_ohlcv("KRW-XRP", interval="day", count=365)
Daily_Check_XRP['diff1'] = round(((Daily_Check_XRP['high'] - Daily_Check_XRP['low']) / Daily_Check_XRP['low']) * 100,2)
Daily_Check_XRP['diff2'] = round(((Daily_Check_XRP['open'] - Daily_Check_XRP['close']) / Daily_Check_XRP['open']) * 100,2)
Daily_Check_XRP_Minus = Daily_Check_XRP[Daily_Check_XRP['diff2'] < 0]
print("# XRP 마이너스만..")
print(Daily_Check_XRP_Minus['diff2'].max())
print(Daily_Check_XRP_Minus['diff2'].min())
print(Daily_Check_XRP_Minus['diff2'].mean() * 2) # 엥???? 
XRP_Lower_Value_V = round(Daily_Check_XRP_Minus['diff2'].mean() * 2,1)
############################################################
BTC_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# BTC_Lower_Value = float(-1.5)
BTC_Lower_Value = float(BTC_Lower_Value_V)

ETH_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# ETH_Lower_Value = float(-1.5)
ETH_Lower_Value = float(ETH_Lower_Value_V)

XRP_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# XRP_Lower_Value = float(-1.5)
XRP_Lower_Value = float(XRP_Lower_Value_V)
############################################################

def Current_Coin(Market) :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_Coin_Current = pyupbit.get_current_price(Market) # 현재 BIT가격
        if isinstance(TEMP_Coin_Current, float) :
            check_get_type = 1
            # print("# WHILE - TEMP_Coin_Current : %s" % TEMP_Coin_Current)
            return TEMP_Coin_Current

def Daily_Check(BitCoin) :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        Daily_Check_BitCoin = pyupbit.get_ohlcv(BitCoin, count=5)
        if isinstance(Daily_Check_BitCoin, pd.DataFrame) :
            check_get_type = 1
            # print("# WHILE - TEMP_Coin_Current : %s" % TEMP_Coin_Current)
            return Daily_Check_BitCoin

price_KRW_BTC = Current_Coin("KRW-BTC") # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
price_KRW_ETH = Current_Coin("KRW-ETH") # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # BIT 현재가
price_KRW_XRP = Current_Coin("KRW-XRP")

Min_Call_Price = float(10000)
Min_Call_BTC_Volume = Min_Call_Price / price_KRW_BTC # 10000원을 현재 BIT가로 나눈값
Min_Call_ETH_Volume = Min_Call_Price / price_KRW_ETH # 10000원을 현재 ETH가로 나눈값
Min_Call_XRP_Volume = Min_Call_Price / price_KRW_XRP # 10000원을 현재 XRP가로 나눈값

def __Make_OrderLists(COIN) :

    # print("# Market : %s" % COIN)

    Total_Order = []
    Trade_List_uuid, Trade_List_side, Trade_List_date = [], [], []
    # print("# KRW-BTC 주문리스트 조회")
    for Dist in ('done','cancel') :
        query = {
        'market': COIN,
        'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
        }
        query_string = urlencode(query).encode()
        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()
        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512',
        }
        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}
        res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
        for res_list in range(len(res.json())) :
            Trade_List_uuid.append(res.json()[res_list]["uuid"])
            Trade_List_side.append(res.json()[res_list]["side"])
            Trade_List_date.append(res.json()[res_list]["created_at"])

    Now_DATE = datetime.datetime.today().strftime("%Y-%m-%d")

    # List에 아무것도 안담기는 경우는???
    # print(Trade_List_uuid) # <class 'list'>
    # print(Trade_List_side) # <class 'list'>
    # print(Trade_List_date) # <class 'list'>
    
    Trade_List_date_10, Trade_List_side_10, idx_list = [], [], []
    for x in Trade_List_date :
        Trade_List_date_10.append(x[:10])
    # print(Trade_List_date_10)
    for idx, val in enumerate(Trade_List_date_10) :
        if val == Now_DATE :
           idx_list.append(idx) 
    # print(idx_list)
    for idx, val in enumerate(Trade_List_side) :
        if idx in idx_list :
            # print("%s : %s" % (Trade_List_date_10[idx], val))
            Trade_List_side_10.append(val)
    # print("# 길이 : %s" % len(Trade_List_uuid))
    # print("# 길이 : %s" % len(Trade_List_side))
    # print("# 길이 : %s" % len(Trade_List_date))

    print("# BID Count : %s" % Trade_List_side.count('bid')) # bid : 매수, ask : 매도
    print("# ASK Count : %s" % Trade_List_side.count('ask')) # bid : 매수, ask : 매도
    print("# Real BID Count #")
    print("# BID Count : %s" % Trade_List_side_10.count('bid')) # bid : 매수, ask : 매도

    return Trade_List_side_10.count('bid') # 매수 건수 만 Return

        # {'uuid': '418f0313-e001-4cc3-9dcc-f656dd24f3b2', uuid	주문의 고유 아이디	String
        #  'side': 'ask', side	주문 종류	String
        #  'ord_type': 'market', ord_type	주문 방식	String
        #  'price': None, price	주문 당시 화폐 가격	NumberString
        #  'state': 'done', state	주문 상태	String
        #  'market': 'KRW-BTC', market	마켓의 유일키	String
        #  'created_at': '2021-02-20T10:40:42+09:00', created_at	주문 생성 시간	DateString
        #  'volume': '0.000078', volume	사용자가 입력한 주문 양	NumberString
        #  'remaining_volume': '0.0', remaining_volume	체결 후 남은 주문 양	NumberString
        #  'reserved_fee': '0.0', reserved_fee	수수료로 예약된 비용	NumberString
        #  'remaining_fee': '0.0', remaining_fee	남은 수수료	NumberString
        #  'paid_fee': '2.509884', paid_fee	사용된 수수료	NumberString
        #  'locked': '0.0', locked	거래에 사용중인 비용	NumberString
        #  'executed_volume': '0.000078', executed_volume	체결된 양	NumberString
        #  'trades_count': 1} trade_count	해당 주문에 걸린 체결 수	Integer

def __Make_Put(CALL, COIN, My_Coin, File_Name) :
    # CALL : 조건(BUY/SELL), COIN : 코인명, My_Coin : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
    # My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
    # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()

    # volume : 주문량 (지정가, 시장가 매도 시 필수)
    # price : 주문 가격. (지정가, 시장가 매수 시 필수)
            # ex) KRW-BTC 마켓에서 1BTC당 1,000 KRW로 거래할 경우, 값은 1000 이 된다.
            # ex) KRW-BTC 마켓에서 1BTC당 매도 1호가가 500 KRW 인 경우, 시장가 매수 시 값을 1000으로 세팅하면 2BTC가 매수된다.
            # (수수료가 존재하거나 매도 1호가의 수량에 따라 상이할 수 있음)

    # My_Coin 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
    # print(type(COIN)) # <class 'str'>
    # print(type(My_Coin)) # <class 'str'>

    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    print("# %s : __Make_Put Function Start" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put Function Start\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    print("# %s : %s CALL Volume/Price : %s개 or 원" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), COIN, My_Coin))
    output.write("# %s : %s CALL Volume/Price : %s개 or 원\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), COIN, My_Coin))
    # 시장가 주문은 ord_type 필드를 price or market 으로 설정해야됩니다.
    # Min_Call_Price = df.loc[df['코인'].str.match(COIN),'1회매수액'].to_string()
    Min_Call_Price = df.loc[df['코인'].str.match(COIN),'1회매수액'].values[0]
    # print("# Min_Call_Price Type : %s" % type(Min_Call_Price)) # <class 'str'>
    print("# Min_Call_Price : %s" % Min_Call_Price) # <class 'str'>
    Market_Code = df.loc[df['코인'].str.match(str(COIN)),'코인명'].values[0]
    if CALL == "SELL" :
        Market_Code = df.loc[df['코인'].str.match(str(COIN)),'코인명'].values[0]
        print("# %s : %s 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Market_Code, upbit.get_balance(ticker=Market_Code))) # 특정 코인이나 원화의 잔고만 조회
        output.write("# %s : %s 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Market_Code, upbit.get_balance(ticker=Market_Code))) # 특정 코인이나 원화의 잔고만 조회
        price_KRW_BitCoin = Current_Coin(Market_Code)
        TEMP_Call_Possible = My_Coin * price_KRW_BitCoin # 현재 보유 BIT개수 * 현재 BIT값

        # # 매도 주문의 경우 ord_type을 market로 설정하고 price을 null 혹은 제외해야됩니다.
        # # My_Coin = upbit.get_balance(ticker="KRW-BTC") # 매도할 때는 전체를 매도하니간 굳이 계산없이 전체
        # if COIN == "BTC" :
        #     print("# %s : BIT 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
        #     output.write("# %s : BIT 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
        #     price_KRW_BTC = Current_Coin("KRW-BTC")
        #     # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
        #     # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
        #     # price_KRW_XRP = pyupbit.get_current_price("KRW-XRP") # <class 'float'> ---> XRP가격은 계속 바뀔테니 다시 조회
        #     TEMP_Call_Possible = My_Coin * price_KRW_BTC # 현재 보유 BIT개수 * 현재 BIT값
        # elif COIN == "ETH" :
        #     print("# %s : ETH 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
        #     output.write("# %s : ETH 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
        #     price_KRW_ETH = Current_Coin("KRW-BTC")
        #     TEMP_Call_Possible = My_Coin * price_KRW_ETH # 현재 보유 BIT개수 * 현재 BIT값
        # elif COIN == "XRP" :
        #     print("# %s : XRP 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
        #     output.write("# %s : XRP 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
        #     price_KRW_XRP = Current_Coin("KRW-XRP")
        #     TEMP_Call_Possible = My_Coin * price_KRW_XRP # 현재 보유 XRP개수 * 현재 XRP값
        # elif COIN == "DOGE" :
        #     print("# %s : XRP 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
        #     output.write("# %s : XRP 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
        #     price_KRW_XRP = Current_Coin("KRW-XRP")
        #     TEMP_Call_Possible = My_Coin * price_KRW_XRP # 현재 보유 XRP개수 * 현재 XRP값
        # else :
        #     print("# COIN Error")

        if TEMP_Call_Possible < float(5000) : # 최소금액에 못미치면 그냥 returne
            print("# %s :---> 매도 COIN이 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매도 COIN이 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        else :
            print("# %s : Make call SELL" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : Make call SELL\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            CALL = "ask" # 매도
            ORD_TYPE = "market"
            PRICE = ''
            VOLUME = My_Coin

    # CALL : 조건(BUY/SELL), COIN : 코인명, My_Coin : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
    # My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
    elif CALL == "BUY" :
        My_WON = upbit.get_balance("KRW")         # 보유 현금 조회
        # print("# My_WON Type : %s" % type(My_WON)) # <class 'float'>
        # 매수 주문의 경우 ord_type을 price로 설정하고 volume을 null 혹은 제외해야됩니다.
        if float(Min_Call_Price) < float(5000) : # 최소금액에 못미치면 그냥 return
        # CALL : 조건(BUY/SELL), COIN : 코인명, My_Coin : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
            print("# %s :---> 매수해야되지만 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매수해야되지만 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        elif My_WON < float(5000) :
            print("# %s :---> 매수해야되지만 보유금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매수해야되지만 보유금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        else :
            print("# %s : Make call BUY" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : Make call BUY\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            CALL = "bid" # 매수
            ORD_TYPE = "price"
            VOLUME = ''
            PRICE = Min_Call_Price
    else :
        print("# %s : Make Call ERROR~!" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.write("# %s : Make Call ERROR~!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    Market_Code = df.loc[df['코인'].str.match(str(COIN)),'코인명'].values[0]

    query = {
    'market': Market_Code,
    'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
    'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
    'price': PRICE, # 유닛당 주문 가격
    'ord_type': ORD_TYPE,
    }

    print(query)
        # 주문 타입 (필수) - limit : 지정가 주문, price : 시장가 주문(매수), market : 시장가 주문(매도)
        # identifier : 조회용 사용자 지정값 (선택) String (Uniq 값 사용)
    query_string = urlencode(query).encode()

    print("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    # print(type(query)) # <class 'dict'>
    output.write("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    output.write(str(query))

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, secret_key) # .decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    # print(jwt_token)
    # print(authorize_token)
    # print(headers)
    res = requests.post(server_url + "/v1/orders", params=query, headers=headers)

    # print("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    # output.write("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    # print(type(res.text)) # <class 'str'>

    # print(res.text) # "uuid":"ac1def11-d5f2-45e9-9d57-5fc7b998d92d","side":"bid","ord_type":"price","price":"10000.0","state":"wait","market":"KRW-BTC","created_at":"2021-04-16T23:45:50+09:00","volume":null,"remaining_volume":null,"reserved_fee":"5.0","remaining_fee":"5.0","paid_fee":"0.0","locked":"10005.0","executed_volume":"0.0","trades_count":0}

     # 매수 : {"error":{"message":"주문가능한 금액(BTC)이 부족합니다.","name":"insufficient_funds_ask"}}
     # 매수 : {"uuid":"9ddd3561-1188-4e5a-83fb-5810bb7f0c94","side":"bid","ord_type":"price","price":"6689.8","state":"wait","market":"KRW-BTC","created_at":"2021-02-27T09:00:29+09:00","volume":null,"remaining_volume":null,"reserved_fee":"3.3449","remaining_fee":"3.3449","paid_fee":"0.0","locked":"6693.1449","executed_volume":"0.0","trades_count":0}
    output_res_text = res.text
    output.write("# %s : %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), output_res_text))
    output.write("\n")

    print("# %s : __Make_Put Function Done" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put Function Done\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    time.sleep(3) # ---> 매도하고나서 바로 또 매도하려고 함. (매도했는데 매도하려는 물량이 없어서 최소수량보다 작다는 메세지 출력됨)

    output.close()

def Total_BitCoin_Invest_Check(File_Name) :

    output = codecs.open(File_Name, "a", "utf-8-sig") 

    # print("############################################################")
    # print(pyupbit.get_tickers(fiat="KRW"))
    # print(pyupbit.get_current_price("KRW-BTC"))
    # print(pyupbit.get_current_price(["KRW-BTC", "KRW-XRP"]))
    # print(pyupbit.get_ohlcv("KRW-BTC", count=5)) # default value는 200
    # print("############################################################")

    # ['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC', 'KRW-XRP', 'KRW-ETC', 'KRW-OMG', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-KMD', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-REP', 'KRW-EMC2', 'KRW-ADA', 'KRW-SBD', 'KRW-POWR', 'KRW-BTG', 'KRW-ICX', 'KRW-EOS', 'KRW-TRX', 'KRW-SC', 'KRW-IGNIS', 'KRW-ONT', 'KRW-ZIL', 'KRW-POLY', 'KRW-ZRX', 'KRW-LOOM', 'KRW-BCH', 'KRW-ADX', 'KRW-BAT', 'KRW-IOST', 'KRW-DMT', 'KRW-RFR', 'KRW-CVC', 'KRW-IQ', 'KRW-IOTA', 'KRW-MFT', 'KRW-ONG', 'KRW-GAS', 'KRW-UPP', 'KRW-ELF', 'KRW-KNC', 'KRW-BSV', 'KRW-THETA', 'KRW-EDR', 'KRW-QKC', 'KRW-BTT', 'KRW-MOC', 'KRW-ENJ', 'KRW-TFUEL', 'KRW-MANA', 'KRW-ANKR', 'KRW-AERGO', 'KRW-ATOM', 'KRW-TT', 'KRW-CRE', 'KRW-SOLVE', 'KRW-MBL', 'KRW-TSHP', 'KRW-WAXP', 'KRW-HBAR', 'KRW-MED', 'KRW-MLK', 'KRW-STPT', 'KRW-ORBS', 'KRW-VET', 'KRW-CHZ', 'KRW-PXL', 'KRW-STMX', 'KRW-DKA', 'KRW-HIVE', 'KRW-KAVA', 'KRW-AHT', 'KRW-LINK', 'KRW-XTZ', 'KRW-BORA', 'KRW-JST', 'KRW-CRO', 'KRW-TON', 'KRW-SXP', 'KRW-LAMB', 'KRW-HUNT', 'KRW-MARO', 'KRW-PLA', 'KRW-DOT', 'KRW-SRM', 'KRW-MVL', 'KRW-PCI', 'KRW-STRAX', 'KRW-AQT', 'KRW-BCHA', 'KRW-GLM', 'KRW-QTCON', 'KRW-SSX', 'KRW-META', 'KRW-OBSR', 'KRW-FCT2', 'KRW-LBC', 'KRW-CBK', 'KRW-SAND', 'KRW-HUM', 'KRW-DOGE', 'KRW-STRK', 'KRW-PUNDIX', 'KRW-FLOW', 'KRW-DAWN', 'KRW-AXS', 'KRW-STX']
    # Total_BitCoin = ['KRW-ETC', 'KRW-BCH', 'KRW-BTG', 'KRW-EOS', 'KRW-QTUM', 'KRW-ARDR', 'KRW-BORA', 'KRW-ADA', 'KRW-VET', 'KRW-BTT', 'KRW-LTC']
    Total_BitCoin_Up, Total_BitCoin_Down = [], []

    ############################################################
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    # 계좌 조회 
    Account_BitCoin = []
    for res_check in res.json() :
        time.sleep(1)
        print("# %s, %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), res_check))
        output.write("# %s, %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), res_check))

        # 20210507 07:17:43, {'currency': 'KRW', 'balance': '2663.02102522', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
        # 20210507 07:17:44, {'currency': 'BTC', 'balance': '0.00106283', 'locked': '0.0', 'avg_buy_price': '70660000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        # 20210507 07:17:45, {'currency': 'XRP', 'balance': '73.20096047', 'locked': '0.0', 'avg_buy_price': '2051.89', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        # 20210507 07:17:46, {'currency': 'ETH', 'balance': '0.01764982', 'locked': '0.0', 'avg_buy_price': '4255000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        # 20210507 07:17:47, {'currency': 'DOGE', 'balance': '5878.25190604', 'locked': '0.0', 'avg_buy_price': '812.45', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        ############################################################
        Account_BitCoin.append("KRW-"+res_check['currency'])

    Nothing_BitCoin = list(set(Total_BitCoin) - set(Account_BitCoin))

    # for x in Total_BitCoin :
    #     print("Total Coin : %s" % x)
    # for x in Account_BitCoin :
    #     print("Account Coin : %s" % x)
    # for x in Nothing_BitCoin :
    #     print("Nothing Coin : %s" % x)

    # 과거 5일 평균가격 가져오기
    for BitCoin in Total_BitCoin :

        # print("# BitCoin : %s" % BitCoin)
        # Daily_Check_BitCoin = pyupbit.get_ohlcv(BitCoin, count=5)
        Daily_Check_BitCoin = Daily_Check(BitCoin)
        # print(type(Daily_Check_BitCoin)) # <class 'pandas.core.frame.DataFrame'>
        # It failed JSONDecodeError ---> TypeError: 'NoneType' object is not subscriptable
        Daily_Check_BitCoin['diff1'] = round(((Daily_Check_BitCoin['high'] - Daily_Check_BitCoin['low']) / Daily_Check_BitCoin['low']) * 100,2)
        Daily_Check_BitCoin['diff2'] = round(((Daily_Check_BitCoin['open'] - Daily_Check_BitCoin['close']) / Daily_Check_BitCoin['open']) * 100,2)
        Daily_Check_BitCoin_Minus = Daily_Check_BitCoin[Daily_Check_BitCoin['diff2'] < 0]
        Daily_Check_BitCoin_Plus = Daily_Check_BitCoin[Daily_Check_BitCoin['diff2'] > 0]
        # print("# Bit Coin #######################")
        # print(BitCoin)
        # print("# Plus #######################")
        # print(Daily_Check_BitCoin_Plus)
        # print("# Minus #######################")
        # print(Daily_Check_BitCoin_Minus)
        # print("# Coin 마이너스만..")
        # print(Daily_Check_Coin_Minus['diff2'].max())
        # print(Daily_Check_Coin_Minus['diff2'].min())
        # print(Daily_Check_Coin_Minus['diff2'].mean())
        BitCoin_Lower_Value_V = round(Daily_Check_BitCoin_Minus['diff2'].mean(),1)
        BitCoin_Upper_Value_V = round(Daily_Check_BitCoin_Plus['diff2'].mean(),1)
        # print("# Lower #######################")
        # print(BitCoin_Lower_Value_V)
        # print("# Upper #######################")
        # print(BitCoin_Upper_Value_V)

        # 가격을 못가져오면 (nan) 안사거나 안팔겠다는 의미
        if math.isnan(BitCoin_Lower_Value_V) :
            BitCoin_Lower_Value_V = -1000
        if math.isnan(BitCoin_Upper_Value_V) :
            BitCoin_Upper_Value_V = 1000
        Total_BitCoin_Up.append(BitCoin_Upper_Value_V)
        Total_BitCoin_Down.append(BitCoin_Lower_Value_V)

    print(Total_BitCoin_Up)
    print(Total_BitCoin_Down)

    # 오늘 현재가격 가져화서 매수결정
    for idx, val in enumerate(Total_BitCoin) :
        time.sleep(1)
        if val not in Nothing_BitCoin :
            print("# %s : %s : 이미 매수되있음" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), val))
            output.write("# %s : %s : 이미 매수되있음\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), val))
            continue
        Daily_Check_BitCoin = pyupbit.get_ohlcv(val, count=1)
        # print("# Today Check #######################")
        # print(Daily_Check_BitCoin)
        # Daily_Check_BitCoin['diff2'] = round(((Daily_Check_BitCoin['open'] - Daily_Check_BitCoin['close']) / Daily_Check_BitCoin['open']) * 100,2)
        Daily_Check_BitCoin['diff2'] = round(((Daily_Check_BitCoin['close'] - Daily_Check_BitCoin['open']) / Daily_Check_BitCoin['open']) * 100,2)
        Daily_Check_BitCoin_Diff2 = Daily_Check_BitCoin['diff2'].values[0] # 오늘 등락율

        # print(type(Daily_Check_BitCoin_Diff2)) # <class 'numpy.float64'>
        # print(type(Total_BitCoin_Down[idx])) # <class 'numpy.float64'>

        if Daily_Check_BitCoin_Diff2 < Total_BitCoin_Down[idx] : # 오늘 등락율이 작다
            print("# %s : %s : BUY ---> 현재 : %s < 평균 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Total_BitCoin[idx], Daily_Check_BitCoin_Diff2, Total_BitCoin_Down[idx]))
            output.write("# %s : %s : BUY ---> 현재 : %s < 평균 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Total_BitCoin[idx], Daily_Check_BitCoin_Diff2, Total_BitCoin_Down[idx]))
            print(upbit.buy_market_order(Total_BitCoin[idx], Total_BitCoin_Money))
        else :
            print("# %s : %s : Wait ---> 현재 : %s > 평균 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Total_BitCoin[idx], Daily_Check_BitCoin_Diff2, Total_BitCoin_Down[idx]))
            output.write("# %s : %s : Wait ---> 현재 : %s > 평균 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Total_BitCoin[idx], Daily_Check_BitCoin_Diff2, Total_BitCoin_Down[idx]))

    return Total_BitCoin, Total_BitCoin_Up, Total_BitCoin_Down

# def Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name) :
# def Start_BIT_Trade(File_Name) :
def Start_BIT_Trade(File_Name, Total_BitCoin) :

    infinite_while = 1

    Check_Hour = []
    while infinite_while == 1 :

        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 

        print("# %s : WHILE Start " % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.write("# %s : WHILE Start\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        now_time = datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")

        ############################################################
        # Loop 안에서 새롭게 조회를 해야 한다
        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/accounts", headers=headers)

        for res_check in res.json() :
            time.sleep(1)
            print("# %s, %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), res_check))
            output.write("# %s, %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), res_check))

        # output.write("# res.json #################\n")
        # output.write(res.json())
        # output.write("\n")

        if len(res.json()) < 2 : # 보유자산 1개 뿐임 
            print("# 보유자산 1개 뿐임 (BIT or WON)")
            Temp_Check = res.json()[0]
            print("# 보유자산 1개 = %s만" % Temp_Check['currency'])
            output.write("# 보유자산 1개 = %s만" % Temp_Check['currency'])
            # continue

        # print("# res.json() TYPE : %s" % type(res.json())) # <class 'list'>
        # print(res.json()) # LIST안에 Dictionary 형태
        Check_Currency = []
        for res_index in range(len(res.json())) :
            # print(type(res.json()[res_index])) # <class 'dict'>
            # print(res.json()[res_index]['currency'])
            Check_Currency.append(res.json()[res_index]['currency'])

        print("# Check_Currency")
        print(Check_Currency)
        output.write("# Check_Currency\n")
        for x in Check_Currency :
            output.write("%s " % x) # TypeError: utf_8_encode() argument 1 must be str, not list
        output.write("\n")
        # print(df['코인'].to_list()) # <class 'pandas.core.series.Series'>

        ############################################################
        # DataFrame에 없는 Coin만 매수 (DataFrame - 현재 계좌)
        # Nothing_Currency = list(set(df['코인'].to_list()) ^ set(Check_Currency)) # set은 type이 set
        Nothing_Currency = list(set(df['코인'].to_list()) - set(Check_Currency)) # set은 type이 set

        print("# Nothing_Currency")
        print(Nothing_Currency)
        output.write("# Nothing_Currency\n")
        for x in Nothing_Currency :
            output.write("%s " % x) # TypeError: utf_8_encode() argument 1 must be str, not list
        output.write("\n")

        for Nothing_Buy_Coin in Nothing_Currency :
            if Nothing_Buy_Coin != 'KRW' :
                Min_Call_Price = df.loc[df['코인'].str.match(str(Nothing_Buy_Coin)),'1회매수액'].values[0]
                # My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
                # ---> 보유하지 않은 상태이므로 아래 NULL값으로 할당
                My_Coin = ''
                print("# %s : Nothing Buy ---> %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Nothing_Buy_Coin))
                output.write("# %s : Nothing Buy ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Nothing_Buy_Coin))
                __Make_Put("BUY", Nothing_Buy_Coin, My_Coin, File_Name)
                # CALL : 조건(BUY/SELL), COIN : 코인명, My_Coin : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
                # ---> BUY시에는 Call_Price_Possible이 없어도 된다.

        Today_Check_Coin = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=1)
        # Daily_Check_Coin['diff1'] = round(((Daily_Check_Coin['high'] - Daily_Check_Coin['low']) / Daily_Check_Coin['low']) * 100,2)

        for x in res.json() :

            print("# %s : for x['currency'] ---> %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
            output.write("# %s : for x['currency'] ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))

            ############################################################ 
            # Main IF 문 : DataFrame
            Market = "KRW-" + x['currency']
            # print(Total_BitCoin)
            if x['currency'] != 'KRW' and (Market not in Total_BitCoin):
                print("# %s : for - if x['currency'] ---> %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
                output.write("# %s : for -if x['currency'] ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))

                ############################################################
                # 매수 / 매도 조건
                ############################################################

                # print(df.loc[df['코인'].str.match(str(x['currency'])),'목표수익'].values[0]) # <class 'numpy.float64'>
                # Sell_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'목표수익'].values[0] # ---> 이익실현 조건 : 단순 목표수익으로 실현 (like 1.5)
                Sell_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'Sell_Condition'].values[0] # ---> 이익실현 조건 : 일평균 상승률

                # Buy_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'물타기'].values[0] # 그냥 지정된 값으로 물타기
                Buy_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'Buy_Condition'].values[0] # 과거 평균값으로 물타기
                
                price_KRW = Current_Coin(Market)
                Market_Coin_WON = round(float(x['balance']) * price_KRW, 1)
                My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
                # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()
                Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].values[0]
                Percent_My_Coin_Perc = round((price_KRW - float(x['avg_buy_price'])) / float(x['avg_buy_price']) * 100, 2)
                ############################################################

                # if Percent_My_Coin_Perc >= 100 :
                if Percent_My_Coin_Perc >= Sell_Condition :
                    if x['currency'] == 'DOGE' :
                        if Percent_My_Coin_Perc > float(25) :
                            __Make_Put("SELL", x['currency'], My_Coin/2, File_Name)
                            continue
                # if (Percent_My_Coin_Perc > 1.5) and (Today_GAP2 >= Sell_Condition) : # 조건만족 ---> 이익실현 : 1.5이상이면서 일평균 상승율보다 상승했을 경우
                    print("# %s : %s 조건만족1 (이익실현) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : %s 조건만족1 (이익실현) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    __Make_Put("SELL", x['currency'], My_Coin, File_Name)
                    # 날짜, 이익금으로 기록
                    output1 = codecs.open("BitCoinProfit.txt", "a", "utf-8-sig") 
                    Profit_Won = round((float(x['balance']) * float(price_KRW)) - (float(x['balance']) * float(x['avg_buy_price'])),2)
                    print("# %s, %f, %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Profit_Won, x['currency']))
                    output1.write("# %s, %f, %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Profit_Won, x['currency']))
                    output1.close()
                elif Percent_My_Coin_Perc < Buy_Condition : # 조건만족 : 물타기
                    # print("########### 1")
                    # print(x['currency'])
                    # print(Percent_My_Coin_Perc)
                    # print(type(Percent_My_Coin_Perc))
                    # print("########### 2")
                    # if x['currency'] == 'DOGE' and Percent_My_Coin_Perc < float(-70) : # and 조건이 안되는 거 같다
                    if x['currency'] == 'DOGE' :
                        if Percent_My_Coin_Perc < float(-50) :
                            print("# %s : %s ******  DOGE 물타기 ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                                (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                            output.write("# %s : %s ****** DOGE 물타기 ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                                (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                            Trade_Count = __Make_OrderLists(Market) # ---> Return 값 : bid : 매수, ask : 매도
                            Fixed_Trade_Count = 1
                            if Trade_Count > Fixed_Trade_Count :
                                print("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                                output.write("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                            else :
                                print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                                output.write("# %s : 매수 건 없음\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                                __Make_Put("BUY", x['currency'], My_Coin, File_Name)
                        else :
                            print("# %s : %s ******  DOGE Wait ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                                (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                            output.write("# %s : %s ****** DOGE Wait ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                                (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                        continue
                    else :
                        print("# %s : %s 조건만족1 (물타기) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                        output.write("# %s : %s 조건만족1 (물타기) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                        Trade_Count = __Make_OrderLists(Market) # ---> Return 값 : bid : 매수, ask : 매도

                        # 아래 건수 만큼 매수가 이루어졌다면 매수 금지
                        Fixed_Trade_Count = 3
                        if Trade_Count > Fixed_Trade_Count :
                            print("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                            output.write("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                        else :
                            print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                            output.write("# %s : 매수 건 없음\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                            __Make_Put("BUY", x['currency'], My_Coin, File_Name)
                            time.sleep(30) # ---> Function을 호출하고 되돌아오는 데 시간이 걸린다
                            # 20210508 10:37:18 : 매수 건 없음
                            # 20210508 10:39:53 : __Make_Put Function Start
                            # CALL : 조건(BUY/SELL), COIN : 코인명, My_Coin : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
                            # ---> BUY시에는 Call_Price_Possible이 없어도 된다.
                else :
                    print("# %s : %s 조건1에 안들어온다 WAIT! ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : %s 조건1에 안들어온다 WAIT! ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))

            ############################################################ 
            # Main IF 문 ELIF : Total_BitCoinf
            elif x['currency'] != 'KRW' and (Market in Total_BitCoin):
                print("# %s : for - elif x['currency'] ---> %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
                output.write("# %s : for -elif x['currency'] ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
                Sell_Condition = Total_BitCoin_Sell_Condition
                Buy_Condition = Total_BitCoin_Buy_Condition
                Market = "KRW-" + x['currency']
                price_KRW = Current_Coin(Market)
                Market_Coin_WON = round(float(x['balance']) * price_KRW, 1)
                My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
                # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()
                Min_Call_Price = 10000 # Total_BitCoin일 경우 임시로 물타기
                Percent_My_Coin_Perc = round((price_KRW - float(x['avg_buy_price'])) / float(x['avg_buy_price']) * 100, 2)
                if Percent_My_Coin_Perc >= Sell_Condition :
                    print("# %s : %s 조건만족2 (이익실현) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : %s 조건만족2 (이익실현) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    print(upbit.sell_market_order(Market, x['balance']))
                    # 날짜, 이익금으로 기록
                    output1 = codecs.open("BitCoinProfit.txt", "a", "utf-8-sig") 
                    Profit_Won = round((float(x['balance']) * float(price_KRW)) - (float(x['balance']) * float(x['avg_buy_price'])),2)
                    print("# %s, %f, %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Profit_Won, x['currency']))
                    output1.write("# %s, %f, %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Profit_Won, x['currency']))
                    output1.close()
                elif Percent_My_Coin_Perc < Buy_Condition : # 조건만족 : 물타기
                    print("# %s : %s 조건만족2 (물타기) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : %s 조건만족2 (물타기) ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    Trade_Count_temp = __Make_OrderLists(Market) # ---> Return 값 : bid : 매수, ask : 매도
                    # 아래 건수 만큼 매수가 이루어졌다면 매수 금지
                    Fixed_Trade_Count_temp = 1
                    if Trade_Count_temp > Fixed_Trade_Count_temp :
                        print("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                        output.write("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                    else :
                        print("# %s : 매수 건 없음 ---> Total_BitCoin 매수 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
                        output.write("# %s : 매수 건 없음 ---> Total_BitCoin 매수 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency']))
                        print(upbit.buy_market_order(Market, Total_BitCoin_Money))
                else :
                    print("# %s : %s 조건2에 안들어온다 WAIT! ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : %s 조건2에 안들어온다 WAIT! ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s\n" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
            else :
                print("# %s : # IF 문 종료" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

        Now_Hour = datetime.datetime.today().strftime("%H")

        # print(Check_Hour)
        if Now_Hour not in Check_Hour :
            Check_Hour.append(Now_Hour)
            # print(Check_Hour)
            # shutil.copyfile(File_Name, 'Bitpython.txt')
            with open(File_Name, "r", encoding='utf-8') as f1:
                last_line = f1.readlines()[-55:]
            output_2 = codecs.open("Bitpython.txt", "w", "utf-8-sig") 
            for writing_log in last_line :
                # print("LogTail : %s" % writing_log)
                output_2.write(writing_log)
            output_2.close()
            f1.close()
        else :
            print("# No Update %s ---> Bitpython.txt" % File_Name)

        print("# %s : WHILE Done" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.write("# %s : WHILE Done\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        print("")
        output.write("# %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))

        output.close()

        time.sleep(60)

        Total_BitCoin, Total_BitCoin_Up, Total_BitCoin_Down = Total_BitCoin_Invest_Check(File_Name)

        time.sleep(60)

if __name__ == '__main__' :

    # start_time = "09:00"
    # stop_time = "15:30"

    DateTime = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "D:\\Python\\Log\\UPBit_History"+"_"+DateTime+".txt"
    output = open(File_Name, "a")
    output.write("# -*- coding: utf-8 -*-\n")

    Total_BitCoin, Total_BitCoin_Up, Total_BitCoin_Down = Total_BitCoin_Invest_Check(File_Name)

    ############################################################
    # Main Program
    # Call_Price_Possible, Call_Volume_Possible = Call_Condition(File_Name)
    Start_BIT_Trade(File_Name, Total_BitCoin)
    # Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name)

    # print("############################################################")
    # print(pyupbit.get_tickers(fiat="KRW"))
    # print(pyupbit.get_current_price("KRW-BTC"))
    # print(pyupbit.get_current_price(["KRW-BTC", "KRW-XRP"]))
    # print(ppyupbit.get_ohlcv("KRW-BTC", count=5)) # default value는 200
    # print("############################################################")
    ############################################################

    ############################################################
    # print(upbit.get_balance("KRW-BTC")) # KRW-XRP 조회
    # print(upbit.get_balance("KRW")) # 보유 현금 조회

        코인명    코인         목표수익          물타기      1회매수액    제한총액  Buy_Condition  \
0       KRW   KRW  100000000.0 -100000000.0  100000000     100            0.0   
1   KRW-BTC   BTC          1.5         -1.5      45300  226500           -3.2   
2   KRW-ETH   ETH          1.5         -1.5      45300  226500           -4.2   
3   KRW-XRP   XRP          1.5         -1.5      45300  226500           -7.6   
4  KRW-DOGE  DOGE          1.5         -1.5      15100   75500          -13.1   

   Sell_Condition  
0             0.0  
1             2.3  
2             3.2  
3             5.2  
4             6.9  
##########################
# BTC 마이너스만..
-0.1
-17.05
-6.450526315789473
# ETH 마이너스만..
-0.03
-29.67
-8.420491803278688
# XRP 마이너스만..
-0.53
-54.52
-15.221960784313719
# 20210511 16:24:27, {'currency': 'KRW', 'balance': '2689.44772976', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
# 20210511 16:24:28, {'currency': 'BTC', 'balance': '0.00621685'

# 20210511 16:26:57, {'currency': 'BORA', 'balance': '18.31501831', 'locked': '0.0', 'avg_buy_price': '273', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:26:58, {'currency': 'XRP', 'balance': '215.7087881', 'locked': '0.0', 'avg_buy_price': '1935.48', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:26:59, {'currency': 'ETH', 'balance': '0.06000741', 'locked': '0.0', 'avg_buy_price': '5209355.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:27:00, {'currency': 'DOGE', 'balance': '4994.292641', 'locked': '0.0', 'avg_buy_price': '822.76', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:27:01, {'currency': 'QTUM', 'balance': '0.25539825', 'locked': '0.0', 'avg_buy_price': '39154.53', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:27:02, {'currency': 'ARDR', 'balance': '98.23182711', 'locked': '0.0', 'avg_buy_price': '509', 'avg_buy_price_modified': False, 'unit_currency

# 20210511 16:30:30, {'currency': 'VET', 'balance': '183.82352941', 'locked': '0.0', 'avg_buy_price': '272', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:30:31, {'currency': 'BCH', 'balance': '0.02923121', 'locked': '0.0', 'avg_buy_price': '1710500', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
[5.5, 8.1, 6.2, 7.4, 6.0, 5.7, 3.6, 1.8, 6.3, 4.2, 3.3]
[-7.1, -2.7, -2.1, -1.8, -7.7, -3.6, -4.4, -9.9, -3.6, -1.4, -6.9]
# 20210511 16:30:45 : KRW-ETC : Wait ---> 현재 : -2.42 > 평균 : -7.1
# 20210511 16:30:46 : KRW-BCH : 이미 매수되있음
# 20210511 16:30:47 : KRW-BTG : BUY ---> 현재 : -2.65 < 평균 : -2.1
{'error': {'message': '주문가능한 금액(KRW)이 부족합니다.', 'name': 'insufficient_funds_bid'}}
# 20210511 16:30:48 : KRW-EOS : BUY ---> 현재 : -2.38 < 평균 : -1.8
{'error': {'message': '주문가능한 금액(KRW)이 부족합니다.', 'name': 'insufficient_funds_bid'}}
# 20210511 16:30:49 : KRW-QTUM : 이미 매수되있음
# 20210511 16:30:50 : KRW-ARDR : 이미 매수되있음
# 20210511 16:30:51 : KRW-BORA : 이미 매수되있음
# 20210511 16:30:53

# 20210511 16:35:23 : WHILE Start 
# 20210511 16:35:24, {'currency': 'KRW', 'balance': '2689.44772976', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
# 20210511 16:35:25, {'currency': 'BTC', 'balance': '0.00621685', 'locked': '0.0', 'avg_buy_price': '71418707.72', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:35:26, {'currency': 'BORA', 'balance': '18.31501831', 'locked': '0.0', 'avg_buy_price': '273', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:35:27, {'currency': 'XRP', 'balance': '215.7087881', 'locked': '0.0', 'avg_buy_price': '1935.48', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:35:28, {'currency': 'ETH', 'balance': '0.06000741', 'locked': '0.0', 'avg_buy_price': '5209355.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:35:29, {'currency': 'DOGE', 'balance': '4994.292641', 'locked': '0.0', 'avg_buy_price': '822.76', 'avg_buy_pri

# 20210511 16:38:59, {'currency': 'ARDR', 'balance': '98.23182711', 'locked': '0.0', 'avg_buy_price': '509', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:39:00, {'currency': 'VET', 'balance': '183.82352941', 'locked': '0.0', 'avg_buy_price': '272', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 16:39:01, {'currency': 'BCH', 'balance': '0.02923121', 'locked': '0.0', 'avg_buy_price': '1710500', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# Check_Currency
['KRW', 'BTC', 'BORA', 'XRP', 'ETH', 'DOGE', 'QTUM', 'ARDR', 'VET', 'BCH']
# Nothing_Currency
[]
# 20210511 16:39:01 : for x['currency'] ---> KRW
# 20210511 16:39:01 : # IF 문 종료
# 20210511 16:39:01 : for x['currency'] ---> BTC
# 20210511 16:39:01 : for - if x['currency'] ---> BTC
# 20210511 16:39:02 : BTC 조건1에 안들어온다 WAIT! ---> 수익조건 : 2.3, 물타기 조건 : -3.2, 현재는 -2.38, 내 KRW-BTC는 0.00621685 (433432.6), 시장가격은 69719000.0
# 20210511 16:39:02 : for x['currency'] ---> BORA
# 20210511 16:3

...

# 20210511 22:26:39 : ARDR 조건2에 안들어온다 WAIT! ---> 수익조건 : 10, 물타기 조건 : -25, 현재는 -10.81, 내 KRW-ARDR는 98.23182711 (44597.2), 시장가격은 454.0
# 20210511 22:26:39 : for x['currency'] ---> VET
# 20210511 22:26:39 : for - elif x['currency'] ---> VET
# 20210511 22:26:40 : VET 조건2에 안들어온다 WAIT! ---> 수익조건 : 10, 물타기 조건 : -25, 현재는 -7.35, 내 KRW-VET는 183.82352941 (46323.5), 시장가격은 252.0
# 20210511 22:26:40 : for x['currency'] ---> BCH
# 20210511 22:26:40 : for - elif x['currency'] ---> BCH
# 20210511 22:26:41 : BCH 조건2에 안들어온다 WAIT! ---> 수익조건 : 10, 물타기 조건 : -25, 현재는 -0.56, 내 KRW-BCH는 0.02923121 (49722.3), 시장가격은 1701000.0
# No Update D:\Python\Log\UPBit_History_20210511.txt ---> Bitpython.txt
# 20210511 22:26:41 : WHILE Done

# 20210511 22:27:43, {'currency': 'KRW', 'balance': '2689.44772976', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
# 20210511 22:27:44, {'currency': 'BTC', 'balance': '0.00621685', 'locked': '0.0', 'avg_buy_price': '71418707.72', 'avg_buy

# 20210511 22:32:12, {'currency': 'BTC', 'balance': '0.00621685', 'locked': '0.0', 'avg_buy_price': '71418707.72', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:32:13, {'currency': 'BORA', 'balance': '18.31501831', 'locked': '0.0', 'avg_buy_price': '273', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:32:14, {'currency': 'XRP', 'balance': '215.7087881', 'locked': '0.0', 'avg_buy_price': '1935.48', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:32:15, {'currency': 'ETH', 'balance': '0.06000741', 'locked': '0.0', 'avg_buy_price': '5209355.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:32:16, {'currency': 'DOGE', 'balance': '4994.292641', 'locked': '0.0', 'avg_buy_price': '822.76', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:32:17, {'currency': 'QTUM', 'balance': '0.25539825', 'locked': '0.0', 'avg_buy_price': '39154.53', 'avg_buy_price_modified': False, 'unit_cu

# 20210511 22:36:51, {'currency': 'QTUM', 'balance': '0.25539825', 'locked': '0.0', 'avg_buy_price': '39154.53', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:36:52, {'currency': 'ARDR', 'balance': '98.23182711', 'locked': '0.0', 'avg_buy_price': '509', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:36:53, {'currency': 'VET', 'balance': '183.82352941', 'locked': '0.0', 'avg_buy_price': '272', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:36:54, {'currency': 'BCH', 'balance': '0.02923121', 'locked': '0.0', 'avg_buy_price': '1710500', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
[6.2, 8.1, 6.3, 9.9, 7.4, 6.8, 3.6, 2.2, 4.7, 4.2, 3.6]
[-7.1, -3.2, -2.1, -8.6, -5.5, -3.6, -5.0, -6.0, -5.2, -1.6, -6.9]
# 20210511 22:37:08 : KRW-ETC : Wait ---> 현재 : -4.92 > 평균 : -7.1
# 20210511 22:37:09 : KRW-BCH : 이미 매수되있음
# 20210511 22:37:10 : KRW-BTG : BUY ---> 현재 : -3.04 < 평균 : -2.1
{'error': {'message': '주문가능한 금액(KRW)이

# 20210511 22:41:37 : KRW-ETC : Wait ---> 현재 : -5.57 > 평균 : -7.1
# 20210511 22:41:38 : KRW-BCH : 이미 매수되있음
# 20210511 22:41:39 : KRW-BTG : BUY ---> 현재 : -3.57 < 평균 : -2.1
{'error': {'message': '주문가능한 금액(KRW)이 부족합니다.', 'name': 'insufficient_funds_bid'}}
# 20210511 22:41:40 : KRW-EOS : Wait ---> 현재 : 23.09 > 평균 : -8.9
# 20210511 22:41:41 : KRW-QTUM : 이미 매수되있음
# 20210511 22:41:42 : KRW-ARDR : 이미 매수되있음
# 20210511 22:41:43 : KRW-BORA : 이미 매수되있음
# 20210511 22:41:44 : KRW-ADA : Wait ---> 현재 : 1.68 > 평균 : -5.8
# 20210511 22:41:45 : KRW-VET : 이미 매수되있음
# 20210511 22:41:47 : KRW-BTT : Wait ---> 현재 : 1.23 > 평균 : -1.2
# 20210511 22:41:48 : KRW-LTC : Wait ---> 현재 : -2.79 > 평균 : -6.9
# 20210511 22:42:48 : WHILE Start 
# 20210511 22:42:49, {'currency': 'KRW', 'balance': '2689.44772976', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
# 20210511 22:42:50, {'currency': 'BTC', 'balance': '0.00621685', 'locked': '0.0', 'avg_buy_price': '71418707.72', 'avg_buy_

# 20210511 22:47:19, {'currency': 'BTC', 'balance': '0.00621685', 'locked': '0.0', 'avg_buy_price': '71418707.72', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:47:20, {'currency': 'BORA', 'balance': '18.31501831', 'locked': '0.0', 'avg_buy_price': '273', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:47:21, {'currency': 'XRP', 'balance': '215.7087881', 'locked': '0.0', 'avg_buy_price': '1935.48', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:47:22, {'currency': 'ETH', 'balance': '0.06000741', 'locked': '0.0', 'avg_buy_price': '5209355.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:47:23, {'currency': 'DOGE', 'balance': '4994.292641', 'locked': '0.0', 'avg_buy_price': '822.76', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
# 20210511 22:47:24, {'currency': 'QTUM', 'balance': '0.25539825', 'locked': '0.0', 'avg_buy_price': '39154.53', 'avg_buy_price_modified': False, 'unit_cu